## Testing Access to the document

In [ ]:
# The ID of a sample document.
DOCUMENT_ID = "1ApWa_x2PJqwjSLSBvV_5715-9WzDl41SoOMjb_2DuLg"

In [5]:
# Google_Oauth_Credentials\client_secret_944564052622-39grtjmehc3c8kk9um4pbjtcnmamaash.apps.googleusercontent.com.json
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/documents.readonly"]


def main():
  """Shows basic usage of the Docs API.
  Prints the title of a sample document.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("Google_Oauth_Credentials/token.json"):
    creds = Credentials.from_authorized_user_file("Google_Oauth_Credentials/token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "./client_secret_944564052622-39grtjmehc3c8kk9um4pbjtcnmamaash.apps.googleusercontent.com.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("./token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("docs", "v1", credentials=creds)

    # Retrieve the documents contents from the Docs service.
    document = service.documents().get(documentId=DOCUMENT_ID).execute()

    print(f"The title of the document is: {document.get('title')}")
  except HttpError as err:
    print(err)


if __name__ == "__main__":
  main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=944564052622-39grtjmehc3c8kk9um4pbjtcnmamaash.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A37403%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments.readonly&state=fNgYf2GBYnvevidjeSeQ6IO4DIe4pO&access_type=offline
The title of the document is: T1D - Cleaned E-book 


## Getting the entire content from the google doc

In [6]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES = 'https://www.googleapis.com/auth/documents.readonly'
DISCOVERY_DOC = 'https://docs.googleapis.com/$discovery/rest?version=v1'


def get_credentials():
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time. If the token is deemed invalid, delete it and run this cell again for it to be created again.
  if os.path.exists("./token.json"):
    creds = Credentials.from_authorized_user_file("./token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "./client_secret_944564052622-39grtjmehc3c8kk9um4pbjtcnmamaash.apps.googleusercontent.com.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("./token.json", "w") as token:
      token.write(creds.to_json())
  return creds


def add_current_and_child_tabs(tab, all_tabs):
  """Adds the provided tab to the list of all tabs, and recurses through and
  adds all child tabs.

  Args:
      tab: a Tab from a Google Doc.
      all_tabs: a list of all tabs in the document.
  """
  all_tabs.append(tab)
  if (tab.get('childTabs')):
    for tab in tab.get('childTabs'):
      add_current_and_child_tabs(tab, all_tabs)


def get_all_tabs(doc):
  """Returns a flat list of all tabs in the document in the order they would
  appear in the UI (top-down ordering). Includes all child tabs.

  Args:
      doc: a document.
  """
  all_tabs = []
  # Iterate over all tabs and recursively add any child tabs to generate a
  # flat list of Tabs.
  for tab in doc.get('tabs'):
    add_current_and_child_tabs(tab, all_tabs)
  return all_tabs


def read_paragraph_element(element):
  """Returns the text in the given ParagraphElement.

  Args:
      element: a ParagraphElement from a Google Doc.
  """
  text_run = element.get('textRun')
  if not text_run:
    return ''
  return text_run.get('content')


def read_structural_elements(elements):
  """Recurses through a list of << Structural Elements >> to read a document's text
  where text may be in nested elements.

  Args:
      elements: a list of Structural Elements.
  """
  text = ''
  for value in elements:
    if 'paragraph' in value:
      elements = value.get('paragraph').get('elements')
      for elem in elements:
        text += read_paragraph_element(elem)
    elif 'table' in value:
      # The text in table cells are in nested Structural Elements and tables may
      # be nested.
      markdown_table = ""
      headers = []
      separator = []

      table = value.get('table')

      # Assuming you have already fetched the table rows and cells as per your snippet
      for index, row in enumerate(table.get('tableRows')):
          cells = row.get('tableCells')
          markdown_row = "|"
          for cell in cells:
              # Read the text from each cell; assuming the function `read_structural_elements` returns a string
              cell_text = read_structural_elements(cell.get('content')).strip()
              markdown_row += f" {cell_text} |"
              
              # Setup headers and separators for the first row
              if index == 0:
                  headers.append(cell_text)
                  separator.append('---')

          # Append the row to the markdown table
          if index == 0:
              # Create the header and separator rows only once, for the first data row
              markdown_table += "|" + " | ".join(headers) + "|\n"
              markdown_table += "|" + " | ".join(separator) + "|\n"
          
          markdown_table += markdown_row + "\n"
      text += markdown_table

      # for row in table.get('tableRows'):
      #   cells = row.get('tableCells')
      #   for cell in cells:
      #     text += read_structural_elements(cell.get('content'))
    elif 'tableOfContents' in value:
      # The text in the TOC is also in a Structural Element.
      toc = value.get('tableOfContents')
      text += read_structural_elements(toc.get('content'))
  return text


def main():
  """Uses the Docs API to print out the text of a document."""
  credentials = get_credentials()
  docs_service = build(
      'docs', 'v1', credentials=credentials
  )
  # Fetch the document with all of the tabs populated, including any nested
  # child tabs.
  doc = (
      docs_service.documents()
      .get(documentId=DOCUMENT_ID, includeTabsContent=True)
      .execute()
  )
  all_tabs = get_all_tabs(doc)

  # Print the text from each tab in the document.
  for tab in all_tabs:
    # Get the DocumentTab from the generic Tab.
    document_tab = tab.get('documentTab')
    doc_content = document_tab.get('body').get('content')
    print(read_structural_elements(doc_content))


if __name__ == '__main__':
  main()

Section 1 - Introduction to Type 1 Diabetes
# articles/diabetes-health/type-1-diabetes-an-overview
1.1 Introduction to Type 1 Diabetes 
You may have heard of diabetes very often, and many of you may have even encountered diabetes during routine checkups. Getting diagnosed and spotted with diabetes could be devastating and disturbing for you, me, and anybody; however, panicking and distressing cannot bring about any productive outcome other than inflaming and worsening the condition deteriorating mental health. Instead of hampering mental peacefulness, you must draw in a clear understanding and comprehension of the medical state that has subjugated you or your loved ones. 

A better comprehension of what has stroked you can make you more acclimatized to the diseased state. Such understanding often opens up proactive measures that you can take to combat and mitigate the risk aspects. Understanding diabetes should start with the type of diabetes spotted in you. Not all diabetes subsets ar